In [2]:
import pandas as pd
import numpy as np
import timm
import torchvision.transforms as T

import sys
sys.path.append('../wildlife_tools/')

from features.deep import DeepFeatures
from features.local import SuperPointExtractor, SiftExtractor, AlikedExtractor, DiskExtractor
from similarity.cosine import CosineSimilarity
from similarity.pairwise.loftr import MatchLOFTR
from similarity.pairwise.lightglue import MatchLightGlue
from similarity.pairwise.collectors import CollectCounts, CollectCountsRansac, CollectAll

from data.dataset import WildlifeDataset
from similarity.wildfusion import SimilarityPipeline, WildFusion
from similarity.calibration import IsotonicCalibration

metadata = {'metadata':  pd.read_csv('TestDataset/metadata.csv'), 'root': 'TestDataset'}
transform = T.Compose([T.Resize([224, 224]), T.ToTensor()])
dataset = WildlifeDataset(**metadata, transform=transform)


In [15]:
matchers = [

    SimilarityPipeline(
        matcher = MatchLightGlue(features='superpoint'),
        extractor = SuperPointExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLightGlue(features='aliked'),
        extractor = AlikedExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLightGlue(features='disk'),
        extractor = DiskExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLightGlue(features='sift'),
        extractor = SiftExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLOFTR(pretrained='outdoor'),
        extractor = None,
        transform = T.Compose([
            T.Resize([512, 512]),
            T.Grayscale(),
            T.ToTensor(),
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = CosineSimilarity(),
        extractor = DeepFeatures(
            model = timm.create_model('hf-hub:BVRA/wildlife-mega-L-384', num_classes=0, pretrained=True)
        ),
        transform = T.Compose([
            T.Resize(size=(384, 384)),
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]),
        calibration = IsotonicCalibration()
    ),
]


priority_matcher =  SimilarityPipeline(
    matcher = CosineSimilarity(),
    extractor = DeepFeatures(
        model = timm.create_model('hf-hub:BVRA/wildlife-mega-L-384', num_classes=0, pretrained=True)
    ),
    transform = T.Compose([
        T.Resize(size=(384, 384)),
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]),
)

wildfusion = WildFusion(calibrated_matchers = matchers, priority_matcher = priority_matcher)

In [4]:
wildfusion.fit_calibration(dataset, dataset)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.63s/it]


In [5]:
wildfusion(dataset, dataset)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.17s/it]


array([[0.99822124, 0.28558058, 0.40018908],
       [0.29122856, 0.99799509, 0.0557037 ],
       [0.40112083, 0.11838401, 0.99962776]])

In [6]:
wildfusion(dataset, dataset, B=1)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.93s/it]


array([[0.99861646,       -inf,       -inf],
       [      -inf, 0.9987793 ,       -inf],
       [      -inf,       -inf, 0.9996744 ]], dtype=float32)

In [7]:
wildfusion(dataset, dataset, B=2)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.82s/it]


array([[0.9984537 ,       -inf, 0.40034783],
       [0.29118976, 0.9979655 ,       -inf],
       [0.4013244 ,       -inf, 0.9996744 ]], dtype=float32)